---
title: Discussion 6
subtitle: Random Forests with Imputed Data
description: 'Thursday, February 13th, 2025'
jupyter:
  jupytext:
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.16.4
  kernelspec:
    display_name: EDS 232
    language: python
    name: eds232-env
---


### Introduction

In this week's discussion section, we will use a data with few NAs and intentionally add more NAs to it. We are going to run different imputation strategies on our newly "NA-ed" dataset, and see which performs best. Normally, you would never know how your imputation is actually performing, but this excercise will allow us to look under the hood  a bit at how different imputation strategies perform differently. Once we find which imputation strategy works best, we will run a random forest on both the original data, as well as our newly imputed data. Which do you think will perform better?? 

## Data 

This week, we will be working with mushroom data! This dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/73/mushroom) includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family. Our target variable will be, `poisonous`, a categorical outcome variable classifying the mushroom as poisonous or not. We will include 22 features in our dataset that all relate to mushroom characteristics- such as `cap-cut`, `cap-surface`, `bruises`, and `odor`. 

## Excercise

#### Import Libraries


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from ucimlrepo import fetch_ucirepo 

#### Load Data


In [ ]:
# Fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# Save data as X and y variables
X = mushroom.data.features 
y = np.ravel(mushroom.data.targets)

# Expand dataframe columns and look at view dataframe
pd.set_option('display.max_columns', None)
X.head()

### Encoding Data


In [ ]:
# Factorize all columns
for col in X.columns: 
    X.loc[:, col] = pd.factorize(X[col], sort = True)[0]

# View first few rows of encoded data
X.iloc[0:5, 0:5]

### Time to impute! 

Does our dataset have any missing values? Lets check! 


In [ ]:
# Check for NAs
X.isna().sum()

We are going to randomly assign observations in our dataset to be missing, and then see which imputation methods perform best by comparing their results to our actual dataset. Let's randomly assign NA observations throughout our dataset. We will create a copy of our dataframe and call it `X_Na`.


In [ ]:
# Create copy of X variables
X_Na = X.copy()

In [ ]:
# Assign 10% of new dataframe with NA values
for col in X_Na.columns: 
    X_Na.loc[X_Na.sample(frac = 0.1).index, col] = np.nan

In [ ]:
# Check to make sure there are missing values
X_Na.isna().sum()

Now that we have our dataset with missing values, let's impute! 

##### Imputation method #1: Filling NA values with the mode


In [ ]:
# Impute with mode
X_mode_impute = X_Na.fillna(X_Na.mode().iloc[0])

# Check to make sure there are no NAs
X_mode_impute.isna().sum()

##### Imputation method #2: Filling NA values with the median using `SimpleImputer`


In [ ]:
# Impute with median (using SimpleImputer) 
median_impute = SimpleImputer(strategy = 'median')
X_median_impute = median_impute.fit_transform(X_Na)
X_median_impute = pd.DataFrame(X_median_impute, columns = X.columns)

# Check to make sure there are no NAs
X_median_impute.isna().sum()

##### Imputation method #3: Filling NA values with KNN Imputer


In [ ]:
# Impute with KNN Imputer
knn_impute = KNNImputer(n_neighbors = 20)
X_knn_impute = knn_impute.fit_transform(X_Na)
X_knn_impute = pd.DataFrame(X_knn_impute, columns = X_Na.columns)

# Check to make sure there are no NAs
X_knn_impute.isna().sum()

##### Now that we have four different dataframes with four different imputation methods, lets see which best captured our real data!We can do this using the mean squared error!  


In [ ]:
# Calculate imputation accuracy using mean squared error
mse_mode = mean_squared_error(X, X_mode_impute)
mse_median = mean_squared_error(X, X_median_impute)
mse_knn = mean_squared_error(X, X_knn_impute)


# Report results
print(f"Mode imputation performance: {mse_mode}")
print(f"Median Imputation performance: {mse_median}")
print(f"KNN Imputation performance: {mse_knn}")

In [ ]:
# Calculate imputation accuracy using R2
r2_mode = r2_score(X, X_mode_impute)
r2_median = r2_score(X, X_median_impute)
r2_knn = r2_score(X, X_knn_impute)


# Report results
print(f"Mode imputation performance: {r2_mode}")
print(f"Median Imputation performance: {r2_median}")
print(f"KNN Imputation performance: {r2_knn}")

It looks like our KNN Imputation was the most successfull in imputing NAs! Let's run a random forest with our actual data, and our KNN imputed data to see how/if they differ! 

### Random Forest Classifier with original data


In [ ]:
# Split actual data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
# Number of features to include for tuning
num_features = [1,4,7,10,13,16,19,22]
accuracy = []

for feature in num_features: 
    rf_classifier = RandomForestClassifier(
        n_estimators = 50, 
        max_depth = 3, 
        random_state = 42, 
        max_features = feature
    )
    
    # Train model
    rf_classifier.fit(X_train, y_train)
    
    # Predict and evaluate 
    y_pred = rf_classifier.predict(X_test)
    rf_accuracy = accuracy_score(y_test, y_pred)
    accuracy.append(rf_accuracy)
    print(f"Number of features: {feature}; Random Forest Accuracy: {rf_accuracy}")

### Random Forest Classifier with imputed data:


In [ ]:
# Split imputed data 
X_train, X_test, y_train, y_test = train_test_split(X_knn_impute, y, test_size = 0.3, random_state = 42)

In [ ]:
# Number of features to include for tuning
# Number of features to include for tuning
num_features = [1,4,7,10,13,16,19,22]
accuracy = []

for feature in num_features: 
    rf_classifier = RandomForestClassifier(
        n_estimators = 50, 
        max_depth = 3, 
        random_state = 42, 
        max_features = feature
    )
    
    # Train model
    rf_classifier.fit(X_train, y_train)
    
    # Predict and evaluate 
    y_pred = rf_classifier.predict(X_test)
    rf_accuracy = accuracy_score(y_test, y_pred)
    accuracy.append(rf_accuracy)
    print(f"Number of features: {feature}; Random Forest Accuracy: {rf_accuracy}")